In [1]:
import os
import sys
#Go back one directory to have access to dependencies
#os.chdir(r"..\\")
#Import local modules
print(os.getcwd())
sys.path.append(os.getcwd())
from ipynb.fs.full.Dependencies.Training_Game import Training_Game

#Standard imports
import numpy as np
import random

#Pygame
import pygame
import pygame.freetype

#For image conversion
from PIL import Image

pygame 2.0.1 (SDL 2.0.14, Python 3.6.10)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
#Core Space invaders game mode.
class Space_Invaders():
    def __init__(self, width, height, screen, surface, mode, scale, player_rockets = 10.0, 
                 enemy_rockets = -3.0, player_speed = 2.0, game_intensity = [0.125, 0.14,0.19,0.9,2.0]):
        #Pygame initialisation variables
        pygame.init()
        self.GAME_FONT = pygame.freetype.Font("Dependencies/Resources/COMIC.TTF", 24)

        #700 and 600 by standard, scales according to this aspect ratio
        self.width = width * scale
        self.height = height * scale
        
        self.true_width = width
        self.true_height = height
        
        self.screen = pygame.display.set_mode((int(width), int(height)), pygame.DOUBLEBUF)
        
        self.clock = pygame.time.Clock()
        
        #Game state check
        self.done = False
        self.lost = False
        self.training_mode = mode
        
        #Standard values with scaling factor
        self.scale = scale
        self.enemySpeed = game_intensity[0] * scale
        self.movement_speed = player_speed * scale
        self.rocket_speed = player_rockets * scale
        self.enemy_rocket_speed = enemy_rockets * scale
        self.intensity = game_intensity
        
        #Enemy general variables
        self.enemyShotTimer = random.uniform(0.5, 10.0)
        self.enemyRockets = []
        self.rockets = []
        self.potentialShooters = []
        self.aliens = []
        self.generator = Generator(self)
        self.max_aliens = len(self.aliens)
        
        #Misc
        player_size = 30
            
        self.player = Invaders_Player(self, self.true_width/2,
                                      self.true_height-(self.true_height/100 * 10),
                                      player_size)

        self.current_reward = 0
        self.render_delay = 10
        
        #If in a normal game, start the standard game loop, otherwise this will be controlled
        #by the space invader env super-class.
        if self.training_mode == False:
            while not self.done:
                self.update()
                self.render()

            print("exiting")
            pygame.display.quit()
            pygame.quit()
            sys.exit()
            
    def execute_action(self, action):
        if action == 0:
            self.move_left()
        if action == 1:
            self.move_right()
        if action == 2:
            self.shoot()
                
    def move_left(self):
        self.player.sprite.rect.x -=self.movement_speed if \
        (self.player.sprite.rect.x / self.scale) > 0 else 0

    def move_right(self):
        boundary =  self.true_width - self.player.size * self.scale
        if self.player.sprite.rect.x < boundary:
            self.player.sprite.rect.x +=self.movement_speed

    def shoot(self):
        #Centre of the player (32*32 by standard)
        origin_point = 16 * self.scale
        if self.player.fire_rate <= 0.0:
            print("successfully shot")
            self.rockets.append(Rocket(self, self.player.sprite.rect.x+origin_point,
                                       self.player.sprite.rect.y))
            self.player.fire_rate = 3.0

    def get_state(self, colour = False):
        #TODO:
        # Time difference from processing full colour or greyscale against the time it takes
        #to actually convert to greyscale every frame
        #Transform the image into a greyscale byte-array
        data = pygame.image.tostring(self.screen, 'RGB')
        
        if colour == True:
            img = Image.frombytes('L', (int(self.true_width), int(self.true_height)), data)
        else:
            img = Image.frombytes('RGB', (int(self.true_width), int(self.true_height)), data)
        #img = img.convert('L')

        #Make 2d
        array2D = np.array(img.getdata())
        array2D.resize((int(self.true_width),int(self.true_height)))
        return array2D
        
    def calculate_reward(self):
        #Return 1 or -1 to indicate a loss or win terminal state
        if self.lost == True:
            return 0
        if self.done and not self.lost or len(self.aliens) == 0:
            return 1
        
        reward = self.current_reward
        self.current_reward = 0
        return reward

    def end_state(self, hasWon, event = None):
        
        if self.training_mode == True:
            print("training instance finished")
            self.done = True
            return
        
        if event == None:
            if hasWon == True:
                self.GAME_FONT.render_to(self.screen, (self.width/2, self.height * 0.2), "Victory", (0, 0, 255))
                self.GAME_FONT.render_to(self.screen, (self.width/2, self.height * 0.35), "Enter to Continue.", (0, 0, 255))
            else:
                self.GAME_FONT.render_to(self.screen, (self.width/2, self.height * 0.2), "Defeat", (0, 0, 255))
                self.GAME_FONT.render_to(self.screen, (self.width/2, self.height * 0.35), "Enter to Continue.", (0, 0, 255))
        else:
            if hasWon == True:
                if event.type == pygame.KEYDOWN and event.key == pygame.K_RETURN:
                    self.done = True
                if event.type == pygame.KEYDOWN and event.key == pygame.K_a:
                    self.reset()
                    return
            elif hasWon == False:
                self.lost = True
                if event.type == pygame.KEYDOWN and event.key == pygame.K_RETURN:
                    self.done = True
                    return
                if event.type == pygame.KEYDOWN and event.key == pygame.K_a:
                    self.reset()
                    return

    
    def clear_arrays(self, arr):
        for a in arr:
            arr.remove(a)
            
    def reset(self):
        #Reset the clock
        self.clock = pygame.time.Clock()
        #Reset the end conditions
        self.hasWon = False
        self.done = False
        self.lost = False
        #Reinitialise all of the values containing game objects
        self.enemyShotTimer = random.uniform(0.5, 10.0)
        
        self.clear_arrays(self.enemyRockets)
        self.clear_arrays(self.rockets)
        self.clear_arrays(self.potentialShooters)
        
        self.enemyRockets = []
        self.potentialShooters = []
        self.aliens = []
        #Generate a new array of aliens and reinitialise the player
        self.generator = Generator(self)
        self.player = Invaders_Player(self, self.width/2, self.height-(self.height/100 * 10), 30)
        
        self.enemySpeed = 0.125 * self.scale
            
        return self.get_state()
    
    def update(self):
         #Check for the victory condition (zero aliens left)
        #print("in update")
        if len(self.aliens) == 0:
            self.end_state(True)
            
        for event in pygame.event.get():
            #Only enable leaving the game for non-training builds
            if len(self.aliens) == 0:
                self.end_state(True, event)
            if event.type == pygame.QUIT:
                self.done = True
            elif event.type == pygame.KEYDOWN and event.key == pygame.K_SPACE:
                if not self.lost and self.player.fire_rate <= 0.0:
                    self.shoot()
                return
            
       # pygame.event.get()
            
        if not self.lost: 
            self.player.update()
            self.player.checkCollision(self)
            
            for alien in self.aliens:
                alien.checkCollision(self)
                #If the aliens have reached the floor, end the game in defeat
                if alien.y > self.height:
                    self.end_state(False)
                    break
                        
            for rocket in self.rockets:
                rocket.update(self.rocket_speed)
            for rocket in self.enemyRockets:
                rocket.update(self.enemy_rocket_speed) 
                
        #Only check and register input if not a training game
        if self.training_mode == False:
            #Get the current pressed buttons and call move functionality
            pressed = pygame.key.get_pressed()
            if pressed[pygame.K_LEFT]:
                self.move_left()
            elif pressed[pygame.K_RIGHT]:
                self.move_right()
                
        #print("update 2")
        #Control Enemy laser fire
        if self.enemyShotTimer <= 0.0:                    
            #Pick a random one of the potential shooters and fire    
            if len(self.enemyRockets) == 0:
                shooter = random.randrange(0, len(self.aliens))
                self.aliens[shooter].fire(self)
                self.enemyShotTimer = random.uniform(2.0, 6.0)
        #Otherwise iterate down the fire cool-down for the aliens.
        else:
            self.enemyShotTimer -= 0.1 * self.scale
        #print("update 3")
        pygame.display.flip()
        self.clock.tick(60)

    def render(self, mode='human'):      
        
        if mode == "human" and self.render_delay <= 0.0:

            #Fill the screen background black
            self.screen.fill((0,0,0))

            #print("render 1")
            #If the player is not dead, draw and check collisions
            if not self.lost: 
                self.player.draw(self)
            else:
                #If the game is just a normal game, display defeat screen and awat prompt
                self.end_state(False)

            #Render the score text to the screen
            if self.training_mode == False:
                self.GAME_FONT.render_to(self.screen, (self.width * 0.1, self.height * 0.9), str(self.player.score), (0, 0, 255))

            #Update the aliens
            if not self.lost: 
                for alien in self.aliens:
                    #Render all the remaining aliens
                    alien.draw(self)

                #Update the existing rockets
                #Render the rockets in each array with a fixed speed.
                for rocket in self.rockets:
                    rocket.draw()
                for rocket in self.enemyRockets:
                    rocket.draw() 
        else:
            self.render_delay -= 0.1
            
#Auxillary classes for the Space invaders game mode
class Generator:
    def __init__(self, game):
        #Reset aliens
        game.aliens = []
        #Set attribute information for each alien including position, spacing and type
        width = 50.0 * game.scale
        margin = 60.0 * game.scale
        size = 30.0 * game.scale       
            
        alienType = 4
        #Initialise iterables
        row = 0
        row_index = 0

        #Cycle through as many times as the width, heigh and margin allow, making new aliens
        for y in range(0, int(int(game.height)/2 - int(width)), int(width)):
            row_index = 0
            for x in range(int(margin), int(game.width) - int(margin), int(width)):
                game.aliens.append(Alien(game, x, y, size,
                                         "Dependencies/Resources/alien" + str(alienType) + ".png",
                                         row, row_index, alienType, game.scale))
                row_index += 1
            #Use a new alien type for each row
            row += 1
            if alienType > 1:
                alienType -= 1
                
class Alien:
    def __init__(self, game, x, y, size, path, row, row_index, alien_type, scale):
        #Position
        self.x = x
        self.y = y
        #Game reference
        self.game = game
        #Position and type reference
        self.row = row
        self.row_index = row_index
        self.alien_type = alien_type
        
        #Misc
        self.size = size * game.scale
        self.alienSprites = pygame.sprite.Group()
        self.sprite = Block((130, 240, 120), self.x, self.y, path, scale)
        self.alienSprites.add(self.sprite)
        
        #Initial move direction (denoting right)
        self.moveDirection = 1
        
        
    def draw(self, game):
        index = 0
        for sprite in self.alienSprites:
            index += 1
            #Update horizontal position
            sprite.velocity[0] = game.enemySpeed
            if self.moveDirection > 0:
                sprite.position[0] = sprite.position[0] + sprite.velocity[0]
            else:
                sprite.position[0] = sprite.position[0] - sprite.velocity[0]
            
            
            boundary = (game.width/game.scale) - game.player.size * game.scale
            
            #If this is the end alien, and it reaches the side, call drop and reverse for all aliens
            #game.width - 30 5
            if sprite.position[0] >= boundary and self.moveDirection == 1 \
            or sprite.position[0] <= 0 and self.moveDirection == -1:
                self.moveDirection = -self.moveDirection

                for alien in game.aliens:
                    alien.drop_and_reverse(self.moveDirection)
            
            #Update sprite position
            sprite.position[1] = sprite.position[1] + sprite.velocity[1]
            sprite.rect.x = sprite.position[0]
            sprite.rect.y = sprite.position[1]
            game.screen.blit(sprite.image, sprite.rect)
            
            #pygame.draw.rect(self.game.screen, (255, 0, 0),
            #    pygame.Rect(sprite.rect.x, sprite.rect.y, self.size, self.size))
    
    def fire(self, game):
        for sprite in self.alienSprites:
            game.enemyRockets.append(Rocket(self.game, sprite.rect.x+16, sprite.rect.y))
        
    def drop_and_reverse(self, newDir):
        #Only update move direction for those who didn't detect the turn
        if self.moveDirection != newDir:
            self.moveDirection = newDir
        #Drop and update sprite position
        for sprite in self.alienSprites:
            sprite.position[1] = sprite.position[1] + 5
            
    def checkCollision(self, game):
        for rocket in game.rockets:
            if rocket.y > game.height or rocket.y < 0:
                game.rockets.remove(rocket)
                
        for sprite in self.alienSprites:
            for rocket in game.rockets:
                if(rocket.x < sprite.rect.x + self.size and
                  rocket.x > sprite.rect.x - self.size and
                  rocket.y < sprite.rect.y + self.size and
                  rocket.y > sprite.rect.y - self.size):
                    game.rockets.remove(rocket)
                    game.aliens.remove(self)
                    game.player.score += 1
                    game.current_reward += 0.1
                    
                    #Update alien speed
                    percentage_remaining = len(game.aliens) / game.max_aliens
                    if percentage_remaining <= 1.0 and percentage_remaining > 0.75:
                        game.enemySpeed = game.intensity[1] 
                    elif percentage_remaining <= 0.75 and percentage_remaining > 0.5:
                        game.enemySpeed = game.intensity[2]
                    elif percentage_remaining <= 0.5 and len(game.aliens) >= 2:
                        game.enemySpeed = game.intensity[3]
                    elif len(game.aliens) < 2:
                        game.enemySpeed = game.intensity[4]
                        
#Class for the rocket for both player and alien.
class Rocket:
    def __init__(self, game, x, y):
        self.x = x
        self.y = y
        self.game = game
    
    def update(self, offset):
        self.y -= offset
        
    def draw(self):
        pygame.draw.rect(self.game.screen, (255, 0, 0),
                        pygame.Rect(self.x, self.y, 2, 4))

#Default container for objects using sprites
class Block(pygame.sprite.Sprite):
    def __init__(self, color, width, height, path, scale):
        super().__init__()
        # Load the image
        self.image = pygame.image.load(path).convert()
        
        #print("size:", self.image.get_rect().size[0])
        self.image = pygame.transform.scale(self.image,
                                            (int(self.image.get_rect().size[0] * scale),
                                             int(self.image.get_rect().size[1] * scale)))
        # Set our transparent color
        self.image.set_colorkey((0,0,0))
        self.rect = self.image.get_rect()
        self.rect.x = width * scale
        self.rect.y = height * scale
        self.position = [width * scale, height * scale]
        self.velocity = [0, 0]
        
class Invaders_Player: 
    def __init__(self, game, x, y, size):
        self.x = x
        self.y = y
        self.size = size
        self.game = game
        self.playerSprites = pygame.sprite.Group()
        self.sprite = Block((130, 240, 120), self.x, self.y, "Dependencies/Resources/paddle.png",
                            game.scale)
        self.playerSprites.add(self.sprite)
        self.fire_rate = 3.0
        self.score = 0
        
    def draw(self, game):
        self.playerSprites.draw(game.screen)

    def update(self):
        if self.fire_rate != 0.0:
            self.fire_rate -=0.1
            if self.fire_rate < 0.0:
                self.fire_rate = 0.0
                
    def checkCollision(self, game):
        if len(game.aliens) > 0:
            for rocket in game.enemyRockets:
                #pygame.draw.rect(self.game.screen, (255, 255, 0),
                #    pygame.Rect(rocket.x, rocket.y, 2, 4))
                if rocket.y > game.height or rocket.y < 0:
                    game.enemyRockets.remove(rocket)

                for sprite in self.playerSprites:
                    #pygame.draw.rect(self.game.screen, (255, 0, 0),
                    #    pygame.Rect(sprite.rect.x, sprite.rect.y, self.size * game.scale, self.size * game.scale))
                    #pygame.draw.line(self.game.screen, (255, 100, 100), 
                     #                (sprite.rect.x, sprite.rect.y), (sprite.rect.x, sprite.rect.y - 5))
                    if(rocket.x < sprite.rect.x + (self.size * game.scale) and
                          rocket.x > sprite.rect.x and
                          rocket.y > sprite.rect.y and
                          rocket.y < sprite.rect.y + self.size * game.scale):
                            if rocket in game.enemyRockets:
                                print("collision detected", rocket.x, sprite.rect.x,  self.size * game.scale)
                                game.enemyRockets.remove(rocket)
                                game.lost = True
                            

In [3]:
#pygame.display.init()
#Space_Invaders(350.0, 300.0, None, None, False, 0.8)

init
init sucessfull
successfully shot
successfully shot
successfully shot
successfully shot
successfully shot
successfully shot
successfully shot
successfully shot
successfully shot
successfully shot
successfully shot
successfully shot
successfully shot
successfully shot
exiting


SystemExit: 

D:\Anaconda\envs\masters\lib\site-packages\IPython\core\interactiveshell.py:3351: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
